In [3]:
# UNCOMMENT IF USING COLAB VM

# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = "CS682/Research"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
# %cd /content/drive/My\ Drive/$FOLDERNAME/cs682/datasets/
# !bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

#!pip install efficientnet_pytorch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/CS682/Research


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
# from efficientnet_pytorch import EfficientNet

import torchvision.datasets as dset
import torchvision.transforms as T
from torchvision import models
import matplotlib.pyplot as plt

from torch.optim import lr_scheduler

import numpy as np
import torch.nn.functional as F
import time
import copy

In [5]:
USE_GPU = True

dtype = torch.half # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [6]:
NUM_TRAIN = 49000
BATCH_SIZE = 64
INPUT_SIZE = 32

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                # T.Resize(INPUT_SIZE)
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./cs682/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=BATCH_SIZE, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs682/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=BATCH_SIZE, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs682/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=BATCH_SIZE)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [7]:
dataloaders = {'train':loader_train, 'val':loader_val,'test':loader_test}
dataset_sizes = { 'train': len(loader_train)*BATCH_SIZE, 'val': len(loader_val)*BATCH_SIZE,'test':len(loader_test)*BATCH_SIZE}

In [8]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')

In [9]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return acc

In [10]:
def train_part34(model, optimizer,scheduler, filename, epochs=10,print_every=100):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    best_acc = float("-inf")
    # train_loss = []
    # train_acc = []
    # val_loss = []
    # val_acc = []
    for e in range(epochs):
        for t, (x, y) in enumerate(dataloaders['train']):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                # train_loss.append(loss.item())
                acc = check_accuracy_part34(dataloaders['val'], model)
                if acc > best_acc:
                  best_acc = acc
                  torch.save(model.state_dict(), filename)

                print()
        scheduler.step()

In [11]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

In [19]:
rotation_net = None

channel_1 = 32
channel_2 = 16
channel_4 = 16
channel_5 = 10
hidden_layer_size = 200
num_classes = 4
num_classes_downstream = 10

H1 = 1 + (32 + 2 * 2 - 5)
W1 = 1 + (32 + 2 * 2 - 5)
H2 = 1 + (H1 + 2 * 2 - 5)
W2 = 1 + (W1 + 2 * 2 - 5)
H3 = 1 + (H2 + 2 * 2*0 - 3)//3
W3 = 1 + (W2 + 2 * 2*0 - 3)//3

H4 = 1 + (H3 + 2 * 1 - 3)
W4 = 1 + (W3 + 2 * 1 - 3)
H5 = 1 + (H4 + 2 * 1 - 3)
W5 = 1 + (W4 + 2 * 1 - 3)
H6 = 1 + (H5 + 2 * 2*0 - 3)//3
W6 = 1 + (W5 + 2 * 2*0 - 3)//3


rotation_net = nn.Sequential(nn.BatchNorm2d(3),nn.ReLU(),nn.Conv2d(3,channel_1,5,padding = 2),
                      nn.BatchNorm2d(channel_1),nn.ReLU(),nn.Conv2d(channel_1,channel_2,5,padding = 2),
                      nn.BatchNorm2d(channel_2), nn.ReLU(),nn.MaxPool2d(3),
                      nn.BatchNorm2d(channel_2),nn.ReLU(),nn.Conv2d(channel_2,channel_4,3,padding = 1),
                      nn.BatchNorm2d(channel_4),nn.ReLU(),nn.Conv2d(channel_4,channel_5,3,padding = 1),
                      Flatten(),nn.Linear(channel_5*H5*W5, hidden_layer_size),
                      nn.Linear(hidden_layer_size, num_classes))

# rotation_net = models.efficientnet_b0(pretrained=False,num_classes = num_classes)

load_model = True
if load_model:
  rotation_net.load_state_dict(torch.load("./rotnet_small_73.pth"))

for param in rotation_net.parameters():
    param.requires_grad = False


rotation_net[17] = nn.Linear(hidden_layer_size,num_classes_downstream)

if torch.cuda.is_available():
    rotation_net.cuda()

In [24]:
learning_rate = 4e-5
momentum = 0.9

criterion = nn.CrossEntropyLoss()

optimizer = optim.RMSprop(rotation_net.parameters(), lr=learning_rate,momentum=momentum)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [25]:
torch.cuda.empty_cache()
train_part34(rotation_net,optimizer, exp_lr_scheduler, "./rotnet_downstream_small.pth")

Iteration 0, loss = 1.6123
Checking accuracy on validation set
Got 444 / 1000 correct (44.40)

Iteration 100, loss = 1.4486
Checking accuracy on validation set
Got 450 / 1000 correct (45.00)

Iteration 200, loss = 1.6973
Checking accuracy on validation set
Got 449 / 1000 correct (44.90)

Iteration 300, loss = 1.5895
Checking accuracy on validation set
Got 450 / 1000 correct (45.00)

Iteration 400, loss = 1.6070
Checking accuracy on validation set
Got 458 / 1000 correct (45.80)

Iteration 500, loss = 1.7813
Checking accuracy on validation set
Got 454 / 1000 correct (45.40)

Iteration 600, loss = 1.6526
Checking accuracy on validation set
Got 447 / 1000 correct (44.70)

Iteration 700, loss = 1.3796
Checking accuracy on validation set
Got 450 / 1000 correct (45.00)

Iteration 0, loss = 1.4561
Checking accuracy on validation set
Got 455 / 1000 correct (45.50)

Iteration 100, loss = 1.5089
Checking accuracy on validation set
Got 440 / 1000 correct (44.00)

Iteration 200, loss = 1.6785
Check

In [26]:
check_accuracy_part34(dataloaders["test"], rotation_net)

Checking accuracy on test set
Got 4585 / 10000 correct (45.85)


0.4585

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(train_loss, '-o')
plt.plot(val_loss, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

train_acc = [x.cpu() for x in train_acc]
val_acc = [x.cpu() for x in val_acc]


plt.subplot(2, 1, 2)
plt.plot(train_acc, '-o')
plt.plot(val_acc, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()